# ACI Assignment 2 - Problem Statement 10

List all the team members BITS ID ,Name along with % of contribution in this assignment:

1. 2025ae05985 Ahamed Imthias 100%
2. 2025ae05984 CHAKRAVADHANULA VINAY KUMAR 100%
3. 2025ae05986 ROUZIF RASHEED MOOPAN 100%
4. 2025ae05987 Y GOWTHAM KUMAR REDDY 100%
5. 2025ae05983 SAKSHAM JINDAL 100%

## Set up

In [1]:
# Imports

import numpy as np

In [2]:
# Configuration

grid_size = (5, 5)  # Grid size for the game
obj_num_cons = 4  # Number of consecutive entries required for win

Data Representation

Grid is represented as a numpy array with the following convention:

0 - Represents empty cell
1 - Represents coin X played by player 1
2 - Represents coin Y played by player 2

## Functions

In [3]:
# Grid manipulation

# Generating grid
def generate_empty_grid(size):

    """
    Generate an empty grid of size size.
    """

    grid = np.zeros(size, dtype=int)

    return grid

# Printing grid
def print_grid(grid):

    """
    Print the grid given
    """

    nrows = grid.shape[0]
    ncols = grid.shape[1]

    column_header = '  '
    for col in range(ncols):
        column_header += f' {col + 1}  '

    print(column_header)
    print('  ' + '___ ' * ncols)
    for row in range(nrows):
        line = f'{row + 1}|'
        for col in range(ncols):
            if grid[row, col] == 0:
                print_char = '_'
            elif grid[row, col] == 1:
                print_char = 'X'
            elif grid[row, col] == 2:
                print_char = 'T'
            else:
                raise Exception('Invalid grid!')
            line += f'_{print_char}_|'
            # print(f'_{print_char}_|')
        print(line)


    return None

def goal_check(grid, player_num):

    """
    Checks if goal has been reached for player 1 or 2.
    """

    if player_num not in [1, 2]:
        raise Exception('Invalid player number!')

    if grid.max() > 2 | grid.min() < 0:
        raise Exception('Invalid grid!')

    nrows = grid.shape[0]
    ncols = grid.shape[1]

    goal_check = False
    for row in range(nrows):
        for col in range(ncols):
            if grid[row, col] != player_num:
                continue
            else:
                # Checking vertical down
                if row + obj_num_cons <= nrows:
                    cells_to_check = [grid[row + i, col] for i in range(obj_num_cons)]
                    if max(cells_to_check) == player_num & min(cells_to_check) == player_num:
                        goal_check = True
                # Checking horizontal right
                if col + obj_num_cons <= ncols:
                    cells_to_check = [grid[row, col + i] for i in range(obj_num_cons)]
                    if max(cells_to_check) == player_num & min(cells_to_check) == player_num:
                        goal_check = True
                # Checking diagonal up right
                if (col + obj_num_cons <= ncols) & (row >= obj_num_cons - 1):
                    cells_to_check = [grid[row - i, col + i] for i in range(obj_num_cons)]
                    if max(cells_to_check) == player_num & min(cells_to_check) == player_num:
                        goal_check = True
                # Checking diagonal down right
                if (col + obj_num_cons <= ncols) & (row + obj_num_cons <= nrows):
                    cells_to_check = [grid[row + i, col + i] for i in range(obj_num_cons)]
                    if max(cells_to_check) == player_num & min(cells_to_check) == player_num:
                        goal_check = True

    return goal_check

def get_static_evaluation_function(grid, player_num):

    """
    Get statis evaluation function.
    """

    score_p1 = get_potential_solves(grid, 1)
    score_p2 = get_potential_solves(grid, 2)

    score = (score_p1 - score_p2) if (player_num == 1) else (score_p2 - score_p1)

    return score

def get_potential_solves(grid, player_num):

    """
    Get number of potential solves.
    """

    if player_num not in [1, 2]:
        raise Exception('Invalid player number!')

    if grid.max() > 2 | grid.min() < 0:
        raise Exception('Invalid grid!')

    nrows = grid.shape[0]
    ncols = grid.shape[1]

    min_presence = 1
    opposite_player_num = 1 if player_num == 2 else 2

    score = 0
    for row in range(nrows):
        for col in range(ncols):
            if grid[row, col] == opposite_player_num:
                continue
            else:
                # Checking vertical down
                if row + obj_num_cons <= nrows:
                    cells_to_check = [grid[row + i, col] for i in np.arange(0, obj_num_cons)]
                    score += score_cells(cells_to_check, player_num)
                # Checking horizontal right
                if col + obj_num_cons <= ncols:
                    cells_to_check = [grid[row, col + i] for i in np.arange(0, obj_num_cons)]
                    score += score_cells(cells_to_check, player_num)
                # Checking diagonal up right
                if (col + obj_num_cons <= ncols) & (row >= obj_num_cons - 1):
                    cells_to_check = [grid[row - i, col + i] for i in np.arange(0, obj_num_cons)]
                    score += score_cells(cells_to_check, player_num)
                # Checking diagonal down right
                if (col + obj_num_cons <= ncols) & (row + obj_num_cons <= nrows):
                    cells_to_check = [grid[row + i, col + i] for i in np.arange(0, obj_num_cons)]
                    score += score_cells(cells_to_check, player_num)

    # Increase score if goal has been found
    goal_attained = goal_check(grid, player_num)
    score += 100 * nrows * ncols * goal_attained

    return score

def score_cells(cells_to_check, player_num):

    """
    Score a set of cells based on their possibility to get to a goal.
    """

    heuristic = {}
    for i in range(obj_num_cons + 1):
        heuristic[i] = pow(i, 2)

    opposite_player_num = 1 if player_num == 2 else 2

    solution_possible = opposite_player_num not in cells_to_check
    num_cells_present = sum(np.array(cells_to_check) == player_num)

    score = solution_possible * heuristic[num_cells_present]

    return score

def get_next_grid_states(grid, player_num):

    """
    Gets the next grid states for a player.
    """

    nrows = grid.shape[0]
    ncols = grid.shape[1]

    next_states = []

    for row in range(nrows):
        for col in range(ncols):
            if grid[row, col] == 0:
                next_state_iter = grid.copy()
                next_state_iter[row, col] = player_num
                next_states.append(next_state_iter)

    return next_states

# Testing functions

sample_grid = generate_empty_grid(grid_size)


sample_grid[3, 3] = 1
sample_grid[1, 2] = 1
sample_grid[2, 1] = 2
sample_grid[3, 0] = 2
sample_grid[0, 3] = 1
sample_grid[1, 3] = 2
sample_grid[0, 4] = 2
sample_grid[3, 2] = 1

print_grid(sample_grid)

print('Goal check: ', goal_check(sample_grid, 2))
print('Score P1: ', get_static_evaluation_function(sample_grid, 1))

next_states = get_next_grid_states(sample_grid, 1)
print('Next states: ')
for state in next_states:
    print_grid(state)

   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|___|___|___|_X_|_T_|
2|___|___|_X_|_T_|___|
3|___|_T_|___|___|___|
4|_T_|___|_X_|_X_|___|
5|___|___|___|___|___|
Goal check:  False
Score P1:  5
Next states: 
   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|_X_|___|___|_X_|_T_|
2|___|___|_X_|_T_|___|
3|___|_T_|___|___|___|
4|_T_|___|_X_|_X_|___|
5|___|___|___|___|___|
   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|___|_X_|___|_X_|_T_|
2|___|___|_X_|_T_|___|
3|___|_T_|___|___|___|
4|_T_|___|_X_|_X_|___|
5|___|___|___|___|___|
   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|___|___|_X_|_X_|_T_|
2|___|___|_X_|_T_|___|
3|___|_T_|___|___|___|
4|_T_|___|_X_|_X_|___|
5|___|___|___|___|___|
   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|___|___|___|_X_|_T_|
2|_X_|___|_X_|_T_|___|
3|___|_T_|___|___|___|
4|_T_|___|_X_|_X_|___|
5|___|___|___|___|___|
   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|___|___|___|_X_|_T_|
2|___|_X_|_X_|_T_|___|
3|___|_T_|___|___|___|
4|_T_|___|_X_|_X_|___|
5|___|___|_

In [5]:
# Game playing function

def play_tic_tac_toe(size):

    """
    Play Tic-Tac-Toe
    """

    print('Welcome to Tic-Tac-Toe!')

    # Generate empty grid
    current_grid = generate_empty_grid(size)
    nrows = current_grid.shape[0]
    ncols = current_grid.shape[1]

    # Print starting grid
    print('Current grid: ')
    print_grid(current_grid)

    # Input whether user plays first or second
    turn_choice = input('Do you want to play first? (y/n): ')
    if turn_choice not in ['n', 'y']:
        raise Exception('Invalid input! Enter either "n" or "y"!')
    user_player_num = 1 if turn_choice == 'y' else 2
    program_player_num = 2 if turn_choice == 'y' else 1

    # Game iteration
    end_of_game = False
    turn_num = 0
    while not end_of_game:

        turn_num += 1

        # User's turn
        if turn_num > 1 or turn_choice == 'y':

            # Get user's turn
            print('Your turn to play!')
            correct_turn = False
            while not correct_turn:

                # Getting turn input
                turn_str = input('Enter your turn in "row_number column_number" format: ')
                turn = list(turn_str.split(' '))
                turn_entry_format_valid = (len(turn) == 2) and all(s.isdigit() for s in turn)
                if not turn_entry_format_valid:
                    print('Invalid input! Enter your turn in "row_number column_number" format: ')
                    continue
                turn_row_num = int(turn[0])
                turn_col_num = int(turn[1])
                # turn_row_num = int(input('Enter row number: '))
                # turn_col_num = int(input('Enter column number: '))

                # Checking turn input
                turn_in_bounds = (turn_row_num >= 1) and (turn_row_num <= nrows) and (turn_col_num >= 1) and (turn_col_num <= ncols)
                turn_in_empty_cell = (current_grid[turn_row_num - 1, turn_col_num - 1] == 0)
                correct_turn = turn_in_bounds and turn_in_empty_cell and turn_entry_format_valid
                if not turn_in_bounds:
                    print(f'Invalid input! Row number should be between 1 and {nrows}. Column number should be between 1 and {ncols}.')
                if not turn_in_empty_cell:
                    print(f'Invalid input! Please enter row and column corresponding to an empty cell!')

            # Update grid
            current_grid[turn_row_num - 1, turn_col_num - 1] = user_player_num
            print('Turn recorded. Current grid: ')
            print_grid(current_grid)

            # Goal check
            user_goal_reached = goal_check(current_grid, user_player_num)
            num_empty_cells = sum(sum(current_grid == 0))
            if user_goal_reached:
                end_of_game = True
                print('You won! Congratulations!')
                break
            if not user_goal_reached and (num_empty_cells == 0):
                end_of_game = True
                print('Match drawn! Thanks for playing!')

        # Program's turn

        # Get list of next states
        next_states = get_next_grid_states(current_grid, program_player_num)
        if next_states == []:
            end_of_game = True
            break

        # Find best next state
        num_next_states = len(next_states)
        best_next_state = next_states[0]
        best_program_score = -10000
        for next_state in next_states:

            # Get all potential user move states
            user_move_states = get_next_grid_states(next_state, user_player_num)

            # Find best score for user (least for program)
            best_user_score = -10000
            program_score_for_user_move = -10000
            for user_move in user_move_states:
                user_score = get_static_evaluation_function(user_move, user_player_num)
                program_score = get_static_evaluation_function(user_move, program_player_num)
                if user_score > best_user_score:
                    best_user_score = user_score
                    program_score_for_user_move = program_score

            # Update program move
            if program_score_for_user_move > best_program_score:
                best_program_score = program_score_for_user_move
                best_next_state = next_state

        # Update program's move
        current_grid = best_next_state
        print('Program has played. Current grid: ')
        print_grid(current_grid)

        # Goal check
        program_goal_reached = goal_check(current_grid, program_player_num)
        num_empty_cells = sum(sum(current_grid == 0))
        if program_goal_reached:
            end_of_game = True
            print('Game over! You have lost!')
            break
        if not program_goal_reached and (num_empty_cells == 0):
            end_of_game = True
            print('Match drawn! Thanks for playing!')

        # Prompt to continue playing
        # continue_game = input('Do you want to continue game? Enter y to continue.')
        # if continue_game == 'y':
        #     end_of_game = False
        # else:
        #     print('Ending game! Thank you for playing!')
        #     end_of_game = True

    return None

play_tic_tac_toe(grid_size)

Welcome to Tic-Tac-Toe!
Current grid: 
   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|___|___|___|___|___|
2|___|___|___|___|___|
3|___|___|___|___|___|
4|___|___|___|___|___|
5|___|___|___|___|___|
Your turn to play!
Turn recorded. Current grid: 
   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|___|___|___|___|___|
2|___|___|___|___|___|
3|___|___|_X_|___|___|
4|___|___|___|___|___|
5|___|___|___|___|___|
Program has played. Current grid: 
   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|___|___|___|___|___|
2|___|_T_|___|___|___|
3|___|___|_X_|___|___|
4|___|___|___|___|___|
5|___|___|___|___|___|
Your turn to play!
Turn recorded. Current grid: 
   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|___|___|___|___|___|
2|___|_T_|_X_|___|___|
3|___|___|_X_|___|___|
4|___|___|___|___|___|
5|___|___|___|___|___|
Program has played. Current grid: 
   1   2   3   4   5  
  ___ ___ ___ ___ ___ 
1|___|___|___|___|___|
2|___|_T_|_X_|___|___|
3|___|___|_X_|___|___|
4|___|___|_T_|___|___|
5|___|___|_